In [ ]:

import random
import pyspark
from pyspark.sql import SparkSession, functions
import ConnectionConfig as cc
from pyspark.sql.functions import *
from pyspark.sql import Row

# Set up environment
cc.setupEnvironment()
cc.listEnvironment()
# Start local Spark cluster
spark = cc.startLocalCluster("locks_dim")
spark.getActiveSession() # To get the active Spark session
df_operational_locks_dim = (spark.read
    .format("jdbc")
    .option("driver", cc.get_Property("driver")) 
    .option("url", cc.create_jdbc()) 
    .option("dbtable", "locks")
    .option("user", cc.get_Property("username")) 
    .option("password", cc.get_Property("password")) 
    .load())

In [98]:
##Extract

df_operational_locks = (df_operational_locks_dim
                        .withColumnRenamed("stationid", "station_id")
                        .withColumnRenamed("lockid", "lock_id").select("station_id", "lock_id"))

# Show the schema and a few records to verify
df_operational_locks.printSchema()
df_operational_locks.show(5)




ALLUSERSPROFILE: C:\ProgramData
APPDATA: C:\Users\dobis\AppData\Roaming
COMMONPROGRAMFILES: C:\Program Files\Common Files
COMMONPROGRAMFILES(X86): C:\Program Files (x86)\Common Files
COMMONPROGRAMW6432: C:\Program Files\Common Files
COMPUTERNAME: VIKI
COMSPEC: C:\WINDOWS\system32\cmd.exe
DRIVERDATA: C:\Windows\System32\Drivers\DriverData
GOPATH: C:\Users\dobis\go
HOMEDRIVE: C:
HOMEPATH: \Users\dobis
IGCCSVC_DB: AQAAANCMnd8BFdERjHoAwE/Cl+sBAAAAAdGjjLHLGEWRGn9vRsCSowQAAAACAAAAAAAQZgAAAAEAACAAAAD7UhTq8CVvkaUfJ5fXTR5kgkvcIed3OfwPabt1yHYIgAAAAAAOgAAAAAIAACAAAABSFZMhyRZv+fj9Q44MNd0sMMQVbnBwNGmcsxiNFFrAcmAAAAAQjo+0swEYFhn4kypkFiEe0Z+EUeRh+XkMWaxY6J5h885R6WUpGPQjsBjQtDBiDzTJJu/Eu8HKO9rNDQ2HtHCLXjOrbciSueB80zvNehaNnexWcFALkN4Q37FMwos4go9AAAAAsBkth/vA4x8SgTkTjgM6mv3GKidgi5oDWFMyb92y29Ab+MuztrSDRGCJMXxOGeO0p8LJ8WZBYOU66GQTsD0WFw==
IPY_INTERRUPT_EVENT: 1292
JPY_INTERRUPT_EVENT: 1292
JPY_PARENT_PID: 2040
JPY_SESSION_NAME: DWH_LockDim.ipynb
LANG: en_US.UTF-8
LANGUAGE: 
LC_ALL: en_US.UTF-8
LOCALAPPDA

In [ ]:
#EXTRACT
df_stations = spark.read \
    .format("jdbc") \
    .option("url", cc.create_jdbc()) \
    .option("driver", cc.get_Property("driver")) \
    .option("dbtable", "stations") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .load()

In [99]:
#Transform
df_operational_stations = (df_stations
                        .withColumnRenamed("stationid", "station_id")
                        .withColumnRenamed("stationnr", "station_nr")
                        .withColumnRenamed("gpscoord", "gps_coord")
                        .select("station_id", "station_nr", "street", "number", "zipcode", "district", "gps_coord")
                       )


# Show the result to verify the selected columns
df_operational_stations.show()


+----------+----------+--------------------+-------+-------+----------+-----------------+
|station_id|station_nr|              street| number|zipcode|  district|        gps_coord|
+----------+----------+--------------------+-------+-------+----------+-----------------+
|         1|       026|         Meir (2000)|     84|   2000| ANTWERPEN|(51.2182,4.41241)|
|         2|       019|          ONTBREKEND|     12|   2000| ANTWERPEN| (51.219,4.40405)|
|         3|       020|Groenkerkhofstraa...|      2|   2000| ANTWERPEN|(51.2187,4.40066)|
|         4|       035|Cockerillkaai (2000)|       |   2000| ANTWERPEN|(51.2104,4.38772)|
|         5|       094|        PALEISSTRAAT|    147|   2018| ANTWERPEN|(51.2047,4.39625)|
|         6|       012|       Singel (2018)|       |   2018| ANTWERPEN|(51.1994,4.39013)|
|         7|       027|Lange Nieuwstraat...|    105|   2000| ANTWERPEN|(51.2195,4.41164)|
|         8|       044|    Belgiëlei (2018)|     99|   2018| ANTWERPEN|(51.2058,4.41836)|
|         

In [100]:
df_operational_locks.show()

+----------+-------+
|station_id|lock_id|
+----------+-------+
|         1|      1|
|         1|      2|
|         1|      3|
|         1|      4|
|         1|      5|
|         1|      6|
|         1|      7|
|         1|      8|
|         1|      9|
|         1|     10|
|         1|     11|
|         1|     12|
|         1|     13|
|         1|     14|
|         1|     15|
|         1|     16|
|         1|     17|
|         1|     18|
|         2|     19|
|         2|     20|
+----------+-------+
only showing top 20 rows



In [101]:
df_operational_stations.show()

+----------+----------+--------------------+-------+-------+----------+-----------------+
|station_id|station_nr|              street| number|zipcode|  district|        gps_coord|
+----------+----------+--------------------+-------+-------+----------+-----------------+
|         1|       026|         Meir (2000)|     84|   2000| ANTWERPEN|(51.2182,4.41241)|
|         2|       019|          ONTBREKEND|     12|   2000| ANTWERPEN| (51.219,4.40405)|
|         3|       020|Groenkerkhofstraa...|      2|   2000| ANTWERPEN|(51.2187,4.40066)|
|         4|       035|Cockerillkaai (2000)|       |   2000| ANTWERPEN|(51.2104,4.38772)|
|         5|       094|        PALEISSTRAAT|    147|   2018| ANTWERPEN|(51.2047,4.39625)|
|         6|       012|       Singel (2018)|       |   2018| ANTWERPEN|(51.1994,4.39013)|
|         7|       027|Lange Nieuwstraat...|    105|   2000| ANTWERPEN|(51.2195,4.41164)|
|         8|       044|    Belgiëlei (2018)|     99|   2018| ANTWERPEN|(51.2058,4.41836)|
|         

In [102]:
#Load
# Save the transformed data to a Delta table
df_joined = df_operational_locks.join(
    df_operational_stations,
    on="station_id",
    how="inner"
)

df_joined.show(5)

+----------+-------+----------+--------------------+------+-------+---------+-----------------+
|station_id|lock_id|station_nr|              street|number|zipcode| district|        gps_coord|
+----------+-------+----------+--------------------+------+-------+---------+-----------------+
|        12|    234|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    233|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    232|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    231|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    230|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
+----------+-------+----------+--------------------+------+-------+---------+-----------------+
only showing top 5 rows



In [103]:
from pyspark.sql.types import Row, StructType, StructField, IntegerType, StringType

In [104]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Define the schema explicitly with all 8 fields
schema = StructType([
    StructField("station_id", IntegerType(), True),
    StructField("lock_id", IntegerType(), True),
    StructField("station_nr", StringType(), True),
    StructField("street", StringType(), True),
    StructField("number", StringType(), True),
    StructField("zipcode", StringType(), True),
    StructField("district", StringType(), True),
    StructField("gps_coord", StringType(), True)
])

# Define the new row data with all 8 fields
new_row_data = Row(station_id=0, lock_id=None, station_nr=None, street="", number=None, zipcode=None, district="", gps_coord="")

# Create a DataFrame with the explicit schema
new_row_df = spark.createDataFrame([new_row_data], schema=schema)

# Union with df_joined (ensure df_joined has the same 8 columns)
df_joined_with_new_row = df_joined.union(new_row_df)

# Show the result
df_joined_with_new_row.show()

+----------+-------+----------+--------------------+------+-------+---------+-----------------+
|station_id|lock_id|station_nr|              street|number|zipcode| district|        gps_coord|
+----------+-------+----------+--------------------+------+-------+---------+-----------------+
|        12|    234|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    233|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    232|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    231|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    230|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    229|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    228|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    227|       120|Schijnpoo

In [105]:
# Filter to find the row with station_id = 302
inserted_row = df_joined_with_new_row.filter(df_joined_with_new_row.station_id == 0)

# Show the result
inserted_row.show()

+----------+-------+----------+------+------+-------+--------+---------+
|station_id|lock_id|station_nr|street|number|zipcode|district|gps_coord|
+----------+-------+----------+------+------+-------+--------+---------+
|         0|   NULL|      NULL|      |  NULL|   NULL|        |         |
+----------+-------+----------+------+------+-------+--------+---------+



In [106]:
# Print schemas to diagnose
print("Schema of df_joined:")
df_joined.printSchema()

print("Schema of new_row_df:")
new_row_df.printSchema()

Schema of df_joined:
root
 |-- station_id: integer (nullable = true)
 |-- lock_id: integer (nullable = true)
 |-- station_nr: string (nullable = true)
 |-- street: string (nullable = true)
 |-- number: string (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- district: string (nullable = true)
 |-- gps_coord: string (nullable = true)

Schema of new_row_df:
root
 |-- station_id: integer (nullable = true)
 |-- lock_id: integer (nullable = true)
 |-- station_nr: string (nullable = true)
 |-- street: string (nullable = true)
 |-- number: string (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- district: string (nullable = true)
 |-- gps_coord: string (nullable = true)



In [107]:
df_joined_with_new_row.write.format("delta").mode("overwrite").saveAsTable("lock_dim")


In [108]:
# Create a temporary view from the Delta table
spark.sql("CREATE OR REPLACE TEMP VIEW lock_dim_view AS SELECT * FROM lock_dim")

DataFrame[]

In [109]:

lock_dim_view = spark.sql("SELECT * FROM lock_dim_view where station_id = 0")
lock_dim_view.show(5)


+----------+-------+----------+------+------+-------+--------+---------+
|station_id|lock_id|station_nr|street|number|zipcode|district|gps_coord|
+----------+-------+----------+------+------+-------+--------+---------+
|         0|   NULL|      NULL|      |  NULL|   NULL|        |         |
+----------+-------+----------+------+------+-------+--------+---------+



In [110]:

lock_dim_view = spark.sql("SELECT * FROM lock_dim_view ")
lock_dim_view.show(5)


+----------+-------+----------+--------------------+------+-------+---------+-----------------+
|station_id|lock_id|station_nr|              street|number|zipcode| district|        gps_coord|
+----------+-------+----------+--------------------+------+-------+---------+-----------------+
|        12|    217|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    218|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    219|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    220|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    221|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
+----------+-------+----------+--------------------+------+-------+---------+-----------------+
only showing top 5 rows



In [111]:
spark.stop()